In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
pop_path = "germany_cities_stadtrecht_2023.csv"
coord_path = "Deutschland_Cities.csv"

pop = pd.read_csv(pop_path)
coords = pd.read_csv(coord_path)

print("pop:", pop.shape)
print("coords:", coords.shape)
pop.head()


pop: (2059, 9)
coords: (612, 9)


,state,city,city_designation,area_km2,postal_code,population_total,male,female,per_km2
0,Berlin,Berlin,NaN,891.12,10178,3782202,1860115,1922087,4 244
1,Hamburg,Hamburg,Freie und Hansestadt,755.09,20038,1910160,936740,973420,2 530
2,Bayern,München,Landeshauptstadt,310.70,80313,1510378,734925,775453,4 861
3,Nordrhein-Westfalen,Köln,NaN,405.02,50667,1087353,527728,559625,2 685
4,Hessen,Frankfurt am Main,NaN,248.31,60311,775790,382226,393564,3 124


In [3]:
df_clean = pop[["state", "city", "population_total"]].copy()
df_clean = df_clean.rename(columns={"population_total": "population"})
df_clean["population"] = pd.to_numeric(df_clean["population"], errors="coerce")

print(df_clean.shape)
df_clean.isna().sum()


(2059, 3)


state         0
city          0
population    0
dtype: int64

In [4]:
def normalize_city_v2(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip().lower()

    # German characters
    s = (s.replace("ä", "a")
           .replace("ö", "o")
           .replace("ü", "u")
           .replace("ß", "ss"))

    # Expand common abbreviations
    s = s.replace(" i.d.", " in der ").replace(" i.d", " in der ")
    s = s.replace(" a. d.", " an der ").replace(" a. d", " an der ")
    s = s.replace(" v. d.", " vor der ").replace(" v. d", " vor der ")

    # Remove parentheses content
    s = re.sub(r"\s*\(.*?\)\s*", " ", s)

    # Keep only part before slash
    s = s.split("/")[0]

    # Remove dots
    s = s.replace(".", " ")

    # Remove punctuation, normalize spaces
    s = re.sub(r"[^a-z0-9\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    return s


In [5]:
df_clean["city_key2"] = df_clean["city"].apply(normalize_city_v2)
coords["city_key2"] = coords["city"].apply(normalize_city_v2)


In [6]:
manual_city_map = {
    "munchen": "munich",
    "koln": "cologne",
    "frankfurt am main": "frankfurt",
    "nurnberg": "nuremberg"
}

df_clean["city_key_final"] = df_clean["city_key2"].replace(manual_city_map)
coords["city_key_final"] = coords["city_key2"]


In [7]:
df_geo = df_clean.merge(
    coords[["city_key_final", "lat", "lng"]],
    on="city_key_final",
    how="inner"
).rename(columns={"lat": "latitude", "lng": "longitude"})

print("Merged:", df_geo.shape)


Merged: (603, 7)


In [8]:
cities = df_geo[["city_key2", "city", "state", "population", "latitude", "longitude"]].copy()

cities = cities.dropna(subset=["latitude", "longitude"])
cities = cities.drop_duplicates(subset=["city_key2"])
cities = cities.reset_index(drop=True)

print("Final N cities:", len(cities))
cities.head()


Final N cities: 575


,city_key2,city,state,population,latitude,longitude
0,berlin,Berlin,Berlin,3782202,52.5167,13.3833
1,hamburg,Hamburg,Hamburg,1910160,53.5500,10.0000
2,munchen,München,Bayern,1510378,48.1372,11.5755
3,koln,Köln,Nordrhein-Westfalen,1087353,50.9422,6.9578
4,frankfurt am main,Frankfurt am Main,Hessen,775790,50.1136,8.6797


In [9]:
cities["demand_weight"] = cities["population"] / cities["population"].sum()
print("Demand sum:", cities["demand_weight"].sum())


Demand sum: 1.0


In [11]:
cities.to_csv("clean_cities_with_population_coords.csv", index=False)


In [12]:
lat = np.radians(cities["latitude"].to_numpy())
lon = np.radians(cities["longitude"].to_numpy())

R = 6371.0
dlat = lat[:, None] - lat[None, :]
dlon = lon[:, None] - lon[None, :]

a = np.sin(dlat/2)**2 + np.cos(lat[:, None]) * np.cos(lat[None, :]) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
D = R * c

N = len(cities)
iu = np.triu_indices(N, k=1)

dist_long = pd.DataFrame({
    "city_a": cities.loc[iu[0], "city"].to_numpy(),
    "city_b": cities.loc[iu[1], "city"].to_numpy(),
    "distance_km": D[iu]
})

print("Distance rows:", dist_long.shape)
dist_long.head()


Distance rows: (165025, 3)


,city_a,city_b,distance_km
0,Berlin,Hamburg,253.700404
1,Berlin,München,503.560156
2,Berlin,Köln,475.698688
3,Berlin,Frankfurt am Main,422.070359
4,Berlin,Stuttgart,510.635691


In [13]:
dist_long.to_csv("distance_all_pairs_uppertriangle.csv", index=False)


In [14]:
# check if the 4 cities exist
target = ["München", "Köln", "Frankfurt am Main", "Nürnberg"]
print(cities[cities["city"].isin(target)][["city","population","latitude","longitude"]])

# distances basic stats
print(dist_long["distance_km"].describe())


                 city  population  latitude  longitude
2             München     1510378   48.1372    11.5755
3                Köln     1087353   50.9422     6.9578
4   Frankfurt am Main      775790   50.1136     8.6797
13           Nürnberg      526091   49.4528    11.0778
count    165025.000000
mean        291.011617
std         150.114212
min           0.000000
25%         174.389844
50%         283.510554
75%         397.434723
max         837.792251
Name: distance_km, dtype: float64
